<h2>In-class transcript from Lecture 4, January 16, 2019</h2>


In [1]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

In [2]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting: whether or not to do partial pivoting, default True
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(LU[piv_col:, piv_col]) 
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [3]:
def Lsolve(L, b):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [4]:
def Usolve(U, y):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
    Output:
      x: the solution vector to U @ x == y
    """
    # Omitted for now, part of hw2
        
    return

In [5]:
def LUsolve(A, b):
    """Solve a linear system Ax = b for x by LU factorization with partial pivoting.
    Parameters: 
      A: the matrix.
      b: the right-hand side
    Outputs (in order):
      x: the computed solution
      rel_res: relative residual norm,
        norm(b - Ax) / norm(b)
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # LU factorization
    L, U, p = LUfactor(A)
    
    # Forward and back substitution
    y = Lsolve(L,b[p])
    x = Usolve(U,y)
    
    # Residual norm
    rel_res = npla.norm(b - A@x) / npla.norm(b)
    
    return (x, rel_res)

In [13]:
A = np.round(20*np.random.rand(5,5))
print('\nA:'); print(A)
xorig = np.round(10*np.random.rand(5))
print('\nxorig:', xorig)
b = A @ xorig
print('\nb:', b)


A:
[[ 2.  3.  8. 15.  5.]
 [ 4.  5. 15. 19. 15.]
 [15. 12. 12. 10.  0.]
 [19. 15. 12. 13. 18.]
 [11.  8. 19. 13.  5.]]

xorig: [7. 7. 3. 0. 7.]

b: [ 94. 213. 225. 400. 225.]


In [17]:
L, U, p = LUfactor(A)

In [15]:
p


array([3, 1, 4, 0, 2])

In [16]:
A[p, :] - L @ U

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [18]:
x, relres = LUsolve(A,b)

In [19]:
x

array([7.00000000e+00, 7.00000000e+00, 3.00000000e+00, 2.30916504e-15,
       7.00000000e+00])

In [20]:
relres

1.45347799685212e-16

In [22]:
A = np.array([[ 4., -1., -1.,  0.],
 [-1.,  4.,  0., -1.],
 [-1.,  0.,  4., -1.],
 [ 0., -1., -1.,  4.]])

In [23]:
A


array([[ 4., -1., -1.,  0.],
       [-1.,  4.,  0., -1.],
       [-1.,  0.,  4., -1.],
       [ 0., -1., -1.,  4.]])

In [24]:
L, U, p = LUfactor(A)

In [26]:
U

array([[ 4.        , -1.        , -1.        ,  0.        ],
       [ 0.        ,  3.75      , -0.25      , -1.        ],
       [ 0.        ,  0.        ,  3.73333333, -1.06666667],
       [ 0.        ,  0.        ,  0.        ,  3.42857143]])

In [27]:
p

array([0, 1, 2, 3])

In [30]:
D= np.diag(np.diag(U))

In [31]:
A - L @ D @ L.T

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [32]:
S = np.sqrt(D)

In [33]:
S

array([[2.        , 0.        , 0.        , 0.        ],
       [0.        , 1.93649167, 0.        , 0.        ],
       [0.        , 0.        , 1.93218357, 0.        ],
       [0.        , 0.        , 0.        , 1.8516402 ]])

In [34]:
LL = L @ S
LL

array([[ 2.        ,  0.        ,  0.        ,  0.        ],
       [-0.5       ,  1.93649167,  0.        ,  0.        ],
       [-0.5       , -0.12909944,  1.93218357,  0.        ],
       [ 0.        , -0.51639778, -0.55205245,  1.8516402 ]])

In [38]:
LL @ LL.T - A


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -5.55111512e-17,
        -2.22044605e-16],
       [ 0.00000000e+00, -5.55111512e-17,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -2.22044605e-16,  0.00000000e+00,
         8.88178420e-16]])

In [36]:
U

array([[ 4.        , -1.        , -1.        ,  0.        ],
       [ 0.        ,  3.75      , -0.25      , -1.        ],
       [ 0.        ,  0.        ,  3.73333333, -1.06666667],
       [ 0.        ,  0.        ,  0.        ,  3.42857143]])